In [ ]:
##policy_evaluation uses Bellman expectation to compute V(s) for that mannual policy.
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=3, suppress=True)

env=gym.make('FrozenLake-v1',render_mode='ansi')
state=env.reset()

print("Initial State:",state)
print("Action Space:",env.action_space)
print("Observation Space:",env.observation_space)
print("Grid shape (rows, cols):",(5,5))
print("Reward range:",(min({r for s in env.unwrapped.P for a in env.unwrapped.P[s] for (_,_,r,_) in env.unwrapped.P[s][a]}),max({r for s in env.unwrapped.P for a in env.unwrapped.P[s] for (_,_,r,_) in env.unwrapped.P[s][a]})))

obs, info = env.reset(seed=42)
frame = env.render()  # returns ANSI text since render_mode='ansi'
print(frame)

gamma=0.99
theta=1e-8
V=np.zeros(env.observation_space.n)

P=env.unwrapped.P
nS=env.observation_space.n    # number of states
nA=env.action_space.n
manual_policy=np.zeros((nS,nA))
manual_policy[:,2]=1.0          # action 2 = RIGHT, prob=1 for all states

def plot(V,policy,col_ramp=1,dpi=175,draw_vals=False):

   # Visualize FrozenLake state values V and policy arrows.

    plt.rcParams['figure.dpi']=dpi
    plt.rcParams.update({'axes.edgecolor':(0.32,0.36,0.38)})
    plt.rcParams.update({'font.size':6 if env.unwrapped.nrow==8 else 8})
    plt.figure(figsize=(3,3))

    desc=env.unwrapped.desc
    nrow,ncol=desc.shape
    V_sq=V.reshape((nrow,ncol))

    plt.imshow(V_sq,cmap='cool' if col_ramp else 'gray',alpha=0.7)
    ax=plt.gca()

    arrow_dict={0:'←',1:'↓',2:'→',3:'↑'}

    for x in range(ncol+1):
        ax.axvline(x-0.5,lw=0.5,color='black')
    for y in range(nrow+1):
        ax.axhline(y-0.5,lw=0.5,color='black')

    for r in range(nrow):
        for c in range(ncol):
            s=r*ncol+c
            val=V[s]
            tile=desc[r,c].decode('utf-8')

            if tile=='H':color='red'
            elif tile=='G':color='green'
            elif tile=='S':color='blue'
            else:color='black'

            plt.text(c,r,tile,ha='center',va='center',color=color,fontsize=10,fontweight='bold')

            if draw_vals and tile not in ['H']:
                plt.text(c,r+0.3,f"{val:.2f}",ha='center',va='center',color='black',fontsize=6)

            if policy is not None:
                best_action=np.argmax(policy[s])
                plt.text(c,r-0.25,arrow_dict[best_action],ha='center',va='center',color='purple',fontsize=12)

    plt.title("FrozenLake: Policy and State Values")
    plt.axis('off')
    plt.show()



# 3) Visualize value function + arrows of this manual policy
plot(V_manual,manual_policy,draw_vals=True)

policy=optimal_policy(env,V_manual,gamma)
print("Optimal policy (one-hot over actions):")
print(policy)

#Means: Action LEFT has probability 1.0 All other actions have probability 0